In [1]:
import re
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import simplejson as json

/Users/azicon/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
res = requests.get("https://piaofang.maoyan.com/dashboard/movie")
res.encoding = 'utf-8'
soup = bs(res.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="mobile" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="initial-scale=1, width=device-width, maximum-scale=1, user-scalable=no" name="viewport"/>
<link href="https://s3plus.meituan.net" rel="dns-prefetch"/>
<link href="https://p0.meituan.net" rel="dns-prefetch"/>
<link href="https://s3plus.meituan.net/v1/mss_e2821d7f0cfe4ac1bf9202ecf9590e67/cdn-prod/file:d6a62569/common.css" rel="stylesheet"/>
<style>
              html, body{
                  max-width: 430px;
                  margin: 0 auto;
              }
              .m-page{
                  display:flex;
                  align-items:center;
                  justify-content:center;
                  flex-direction: column;
                  min-height: 100vh;
              }
              .m-page .hint{
                  background: url(https://s0.meituan.net/bs/file/?f=myfe/piaofang:img/news/comments-none-ccd31fb5.png) no-repeat center .4rem;
                  background-size: 2rem;
 

In [3]:
browserOptions = Options()
#browserOptions.add_argument("--headless")

capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"
capa["goog:loggingPrefs"] = {"performance": "ALL"}
driver = webdriver.Chrome(desired_capabilities=capa)
wait = WebDriverWait(driver, 20)

In [4]:
#create snapshot of the entire page to prevent it from constantly changing
driver.get("https://piaofang.maoyan.com/dashboard/movie")
test = None
while not test:
    try:
        test = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'moviename-td')))
    except:
        driver.refresh();
        
logs_raw = driver.get_log("performance")
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

def log_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "json" in log_["params"]["response"]["mimeType"]
    )

responses = []

for log in filter(log_filter, logs):
    request_id = log["params"]["requestId"]
    resp_url = log["params"]["response"]["url"]
    print(f"Caught {resp_url}")
    response = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
    responses.append(response)
    

    

Caught https://piaofang.maoyan.com/dashboard-ajax/movie?orderType=0&uuid=f3c810db-1b84-494c-83d8-472bc372c6e2&timeStamp=1644625811443&User-Agent=TW96aWxsYS81LjAgKE1hY2ludG9zaDsgSW50ZWwgTWFjIE9TIFggMTBfMTVfNykgQXBwbGVXZWJLaXQvNTM3LjM2IChLSFRNTCwgbGlrZSBHZWNrbykgQ2hyb21lLzk3LjAuNDY5Mi45OSBTYWZhcmkvNTM3LjM2&index=58&channelId=40009&sVersion=2&signKey=1957b82ef0a6d728fe96876995fdf113


In [5]:
body0 = json.loads(responses[0]['body'])
movieList = body0['movieList']['list']
date = body0['calendar']['today']
font_url = body0['fontStyle'].split('"')[-2]
font_url

'//s3plus.meituan.net/v1/mss_73a511b8f91f43d0bdae92584ea6330b/font/2ab02453.woff'

In [ ]:
# Get reference fonts from the file tree
from fontTools.ttLib import TTFont
headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) "
              "Chrome/66.0.3359.139 Safari/537.36 "
    }

woff_url = 'http:' + font_url
response_woff = requests.get(woff_url, headers=headers).content

print("Woff retrieval succuessful: " + str(len(response_woff) > 0))

with open('fonts.woff', 'wb') as f:
    f.write(response_woff)

# Already saved, DO NOT RUN THE FOLLOWING 
#with open('basefonts.woff', 'wb') as f:
    #f.write(response_woff)
    
#font1 = TTFont('basefonts.woff')
#font1.saveXML('basefonts.xml')

In [7]:
import pandas as pd
df = pd.DataFrame.from_records(movieList)

In [14]:
df['boxSplitUnit'][4]

{'num': '&#xe13d;&#xe66a;&#xf581;.&#xe1d4;&#xf581;', 'unit': '万'}

In [16]:
# Create Bijection between numbers and unicode 
unicodeToInt = {"uniF581": 0, "uniF56B": 1, "uniE13D": 2, "uniE13D": 3, "uniEAA8": 4, 
                "uniE808": 5, "uniE1D4": 6, "uniE66A": 7, "uniE43F": 8, "uniECDC": 9}

def hexToUnicode(hexa):
    return 'uni' + hexa[3:].upper()



In [19]:
# Map contours to numbers
baseFonts = TTFont('basefonts.woff')
currentFonts = TTFont('fonts.woff')
glyphToInt = {}

for i in unicodeToInt:
    baseGlyph = baseFonts['glyf'][i]
    glyphToInt[baseGlyph] = unicodeToInt[i]

glyphToInt

2 extra bytes in post.stringData array


{'<fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x7fa76d3a8f28>': 0,
 '<fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x7fa76d3a8dd8>': 1,
 '<fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x7fa76d3a8f98>': 3,
 '<fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x7fa76d3a8470>': 4,
 '<fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x7fa76d3a87f0>': 5,
 '<fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x7fa76d3a8a20>': 6,
 '<fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x7fa76d3a86a0>': 7,
 '<fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x7fa76d3a8fd0>': 8,
 '<fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x7fa76d3a85c0>': 9}

In [10]:
integerLookup = {'&#xe984': 1, '&#xf43b': 2, '&#xf5fe': 3, '&#xea24': 4, '&#xef0d': 5, 
                 '&#xea11': 6, '&#xf3df': 7, '&#xec39': 8, '&#xea1b': 9, '&#xeffa': 0}

unitLookup = {'百': 100, '千': 1000, '万': 10000, '亿': 1*10**8}

#converts the weird character to a float
def convertToFloat(string):
    spCharLst = string.split(';')
    result = ''
    for i in spCharLst:
        if len(i) > 7: #has a dot in front
            result += '.' + str(integerLookup[i[1:]])
        elif len(i) == 7: #in case of bad parsing
            result += str(integerLookup[i])
    return float(result)

#helper function for converting the entire block to a single int
def convertDictToInt(dictionary):
    return int(convertToFloat(dictionary['num']) * unitLookup[dictionary['unit']])


In [11]:
convertDictToInt(df['boxSplitUnit'][0]) #裂开，不是bijection

KeyError: '&#xf56b'

In [ ]:
df['boxSplitUnit'] = df['boxSplitUnit'].apply(convertDictToInt)
df['splitBoxSplitUnit'] = df['splitBoxSplitUnit'].apply(convertDictToInt)
df['movieInfo'] = df['movieInfo'].apply(lambda x : x['movieName'])
df

In [ ]:
#大盘
body0['movieList']['nationBoxInfo']